# Dog Rates Data Wrangling

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#gather">Gathering</a></li>
<li><a href="#assess">Assessing</a></li>
<li><a href="#clean">Cleaning</a></li>
<li><a href="#analyze&visualize">Analyze and Visualize</a></li>
<li><a href="#conclusion">Conclusion</a></li>
</ul>

<a id='intro'></a>
## Introduction

The dataset is the tweet archive of Twitter user @dog_rates, also known as WeRateDogs. WeRateDogs is a Twitter account that rates people's dogs with a humorous comment about the dog.

The data wrangling process will consist of 3 main steps. They are:
1. **Data gathering** - The data can be gathered in many ways including web scraping, using APIs etc. The data can be gathered from a single source or from many different sources.
2. **Assessing the data** - The data needs to assessed for quality and tidiness issues. This can be done visually and/or programatically.
3. **Cleaning the data** - Based on the assessment, the data is cleaned and tested to make sure all the issues identified are resolved.

**Analyze & Visualize**
<br/>
Finally, the wrangled data is analyzed and visualized in an effective and insightful manner.

>*Importing all necessary packages for the data wrangling and analysis*

In [1]:
import requests
import tweepy
import pandas as pd
import time
import json

import config

<a id='gather'></a>
## Gathering

> The data for this analysis is to be gathered from multiple sources. They are:
>
>> 1. The WeRateDogs Twitter archive is enhanced and provided. This file (**twitter_archive_enhanced.csv**) just needs to be downloaded.
>
>> 2. The tweet image predictions, i.e., what breed of dog (or other object, animal, etc.) is present in each tweet according to a neural network. This file (**image_predictions.tsv**) is hosted on Udacity's servers and should be downloaded programmatically.
>
>> 3. Additional required and interesting data is to be obtained by querying the Twitter API for each tweet's JSON data and store each tweet's entire set of JSON data in a file (**tweet_json.txt**).

>*The file containing enhanced twitter archive (twitter_archive_enhanced.csv) has been manually downloaded and is available in the directory. The tweet image predictions file (image_predictions.tsv) is to be downloaded programmatically using the URL provided.*

In [2]:
# storing the URL provided in a variable
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'

# getting the response from the URL using requests library 
response = requests.get(url)

# with keyword ensures that the file is closed immediately the desired operation is complete
# file is opened for writing in binary mode
with open('image_predictions.tsv', 'wb') as file:
    # content of the response is written to the file
    file.write(response.content)

>*The file containing the image predictions is successfully saved in the working directory. Now, this data needs to be stored in a new dataframe for further steps of the data wrangling process. In order to read a TSV using pandas, the separator (sep) should be defined to be '\t'.*

In [3]:
image_predictions_df = pd.read_csv('image_predictions.tsv', sep='\t', index_col=None)
image_predictions_df.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


>*The additional data needs to be downloaded by querying the Twitter API using tweepy library. In order to do that, create a twitter developer account after signing in/up. Once the account is created, the consumer keys and authentication tokens will be available for use.*

>*It is not safe to expose the consumer keys and authentication tokens via code. Hence, a config file can used and imported in this notebook. (In order to execute the rest of the notebook, please fill in the necessary details in the config.py file)*

>*Authenticate using the consumer keys and set the access tokens.* 

In [4]:
# create an OAuthHandler instance
auth = tweepy.OAuthHandler(config.API_KEY, config.API_SECRET_KEY)
# set the access tokens
auth.set_access_token(config.ACCESS_TOKEN, config.ACCESS_TOKEN_SECRET)

# create the API instance
# wait_on_rate_limit – whether or not to automatically wait for rate limits to replenish
# wait_on_rate_limit_notify – whether or not to print a notification when Tweepy is waiting for rate limits to replenish
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
api

>*The API instance is created and ready for use now.*

>*The ID corresponding to each tweet is required in order to access the additional details of the tweet. These IDs are present in the twitter-archive-enhanced.csv file. Read the file and store as dataframe for further use.*

In [5]:
# read the file twitter-archive-enhanced.csv and store it in a dataframe 
twitter_archive_df = pd.read_csv('twitter-archive-enhanced.csv', index_col=None, encoding = 'utf-8')
twitter_archive_df.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [6]:
total_number_of_tweets = len(twitter_archive_df.tweet_id)
number_of_failures = 0
failed_tweets_dict = []

start = time.time()

# opening a text file in write mode and writing the JSON containing additional details of the tweet 
with open('tweet_json.txt', 'w') as txt_file:
    # looping over all the tweets whose IDs are present in the twitter_archive_df dataframe
    for tweet_id in twitter_archive_df.tweet_id:    
        try:
            # get a single status specified by the ID parameter
            # extended tweet mode gives the entire untruncated text of the Tweet
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            json.dump(tweet._json, txt_file)
            txt_file.write('\n')
        except tweepy.TweepError as e:
            number_of_failures += 1
            failed_tweets_dict.append(tweet_id)
            print('Tweet ID:', tweet_id, '-', e)
            continue

print('Total number of tweets:', total_number_of_tweets)
print('Time taken:', (time.time()-start)/60, 'minutes')
print('Total number of failed tweets:', number_of_failures)
print('List of failed tweet IDs:', failed_tweets_dict)

Tweet ID: 888202515573088257 - [{'code': 144, 'message': 'No status found with that ID.'}]
Tweet ID: 873697596434513921 - [{'code': 144, 'message': 'No status found with that ID.'}]
Tweet ID: 872668790621863937 - [{'code': 144, 'message': 'No status found with that ID.'}]
Tweet ID: 872261713294495745 - [{'code': 144, 'message': 'No status found with that ID.'}]
Tweet ID: 869988702071779329 - [{'code': 144, 'message': 'No status found with that ID.'}]
Tweet ID: 866816280283807744 - [{'code': 144, 'message': 'No status found with that ID.'}]
Tweet ID: 861769973181624320 - [{'code': 144, 'message': 'No status found with that ID.'}]
Tweet ID: 856602993587888130 - [{'code': 144, 'message': 'No status found with that ID.'}]
Tweet ID: 851953902622658560 - [{'code': 144, 'message': 'No status found with that ID.'}]
Tweet ID: 845459076796616705 - [{'code': 144, 'message': 'No status found with that ID.'}]
Tweet ID: 844704788403113984 - [{'code': 144, 'message': 'No status found with that ID.'}]

Rate limit reached. Sleeping for: 468


Tweet ID: 754011816964026368 - [{'code': 144, 'message': 'No status found with that ID.'}]
Tweet ID: 680055455951884288 - [{'code': 144, 'message': 'No status found with that ID.'}]


Rate limit reached. Sleeping for: 463


Total number of tweets: 2356
Time taken: 34.66343498627345 minutes
Total number of failed tweets: 25
List of failed tweet IDs: [888202515573088257, 873697596434513921, 872668790621863937, 872261713294495745, 869988702071779329, 866816280283807744, 861769973181624320, 856602993587888130, 851953902622658560, 845459076796616705, 844704788403113984, 842892208864923648, 837366284874571778, 837012587749474308, 829374341691346946, 827228250799742977, 812747805718642688, 802247111496568832, 779123168116150273, 775096608509886464, 771004394259247104, 770743923962707968, 759566828574212096, 754011816964026368, 680055455951884288]


>*Below are the explanations of the status codes for the errors:*<br/>
>>*Status code 144 - Corresponds with HTTP 404. The requested Tweet ID is not found (if it existed, it was probably deleted)*
>
>>*Status code 179 - Corresponds with HTTP 403. Thrown when a Tweet cannot be viewed by the authenticating user, usually due to the Tweet’s author having protected their Tweets.*

>*The additional data corresponding to all the tweets in the dataframe are available in tweet_json.txt file. The next step is to read the file and get the required data from JSONs (corresponding to each of the tweets). Finally, store the data in a new dataframe.*

In [7]:
additional_data = []

# opening the tweet_json.txt file in read mode 
with open('tweet_json.txt', 'r') as infile:
    # looping over each line of the file
    for record in infile:
        # convert string to JSON
        record_json_data = json.loads(record)
        # storing the required additional details in a list and appending it to the additional_data list
        additional_data.append([record_json_data['id'], record_json_data['retweet_count'], record_json_data['user']['favourites_count'], record_json_data['user']['followers_count'], record_json_data['created_at']])

# creating a new dataframe using the additional_data list of lists 
additional_data_df = pd.DataFrame(additional_data)
# defining the column names of the dataframe
additional_data_df.columns = ['tweet_id', 'retweet_count', 'favourites_count', 'followers_count', 'created_at']
additional_data_df.head() 

,tweet_id,retweet_count,favourites_count,followers_count,created_at
0,892420643555336193,7492,145955,8876619,Tue Aug 01 16:23:56 +0000 2017
1,892177421306343426,5559,145955,8876619,Tue Aug 01 00:17:27 +0000 2017
2,891815181378084864,3681,145955,8876619,Mon Jul 31 00:18:03 +0000 2017
3,891689557279858688,7661,145955,8876619,Sun Jul 30 15:58:51 +0000 2017
4,891327558926688256,8273,145955,8876619,Sat Jul 29 16:00:24 +0000 2017


>*Checking the number of records to confirm that all valid records in twitter_archive_df dataframe have a corresponding record in additional_data_df dataframe.*

In [8]:
additional_data_df.shape

(2331, 5)

>*We observe that except for the 25 tweet IDs that are not valid, there is one record for each tweet in the dataframe. The data gathering step is now complete.*

<a id='assess'></a>
## Assessing

>There are 2 types of issues that need to be assessed. They are:
>
>>1. Quality issues - issues with content. Low quality data is also known as dirty data.
>
>>2. Tidiness issues -  issues with structure that prevent easy analysis. Untidy data is also known as messy data. Tidy data requirements: 1.Each variable forms a column  2.Each observation forms a row  3.Each type of observational unit forms a table
>
>These issues can be assessed in 2 ways. They are:
>
>>1. Visual assessment - scrolling through the data in your preferred software application.
>
>>2. Programmatic assessment - using code to view specific portions and summaries of the data.

##### Visual assessement

#### 1. twitter_archive_df

In [9]:
twitter_archive_df.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [10]:
twitter_archive_df.tail()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
2351,666049248165822465,NaN,NaN,2015-11-16 00:24:50 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a 1949 1st generation vulpix. Enj...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666049248...,5,10,None,None,None,None,None
2352,666044226329800704,NaN,NaN,2015-11-16 00:04:52 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a purebred Piers Morgan. Loves to Netf...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666044226...,6,10,a,None,None,None,None
2353,666033412701032449,NaN,NaN,2015-11-15 23:21:54 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is a very happy pup. Big fan of well-main...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666033412...,9,10,a,None,None,None,None
2354,666029285002620928,NaN,NaN,2015-11-15 23:05:30 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a western brown Mitsubishi terrier. Up...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666029285...,7,10,a,None,None,None,None
2355,666020888022790149,NaN,NaN,2015-11-15 22:32:08 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a Japanese Irish Setter. Lost eye...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666020888...,8,10,None,None,None,None,None


In [11]:
twitter_archive_df.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
247,845459076796616705,NaN,NaN,2017-03-25 02:15:26 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: Here's a heartwarming scene of ...,7.562885e+17,4.196984e+09,2016-07-22 00:43:32 +0000,https://twitter.com/dog_rates/status/756288534...,12,10,None,None,None,None,None
679,789137962068021249,NaN,NaN,2016-10-20 16:15:26 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Bo. He's a West Congolese Bugaboop Snu...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/789137962...,12,10,Bo,None,None,None,None
364,829141528400556032,NaN,NaN,2017-02-08 01:35:19 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Malcolm. He goes from sneaky tongue sl...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/829141528...,12,10,Malcolm,None,None,None,None
635,793601777308463104,NaN,NaN,2016-11-01 23:53:02 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Dobby. I can't stop looking at her fee...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/793601777...,12,10,Dobby,None,None,None,None
868,761750502866649088,NaN,NaN,2016-08-06 02:27:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...","RT @dog_rates: ""Tristan do not speak to me wit...",6.853251e+17,4.196984e+09,2016-01-08 05:00:14 +0000,https://twitter.com/dog_rates/status/685325112...,10,10,None,None,None,None,None


>*From the visual assessment performed by scrolling through select records of the dataframe, the following are the issue(s) identified:*
>
>>*Quality issues:*
>
>>> 1) In `twitter_archive_df`, the collowing columns have missing values:
>
>>>>   i. `in_reply_to_status_id`
>
>>>>   ii. `in_reply_to_user_id`
>
>>>>   iii. `retweeted_status_id`
>
>>>>   iv. `retweeted_status_user_id`
>
>>>>   v. `retweeted_status_timestamp`
>
>>> 2) Some dog names are not valid (Eg. a, None)
>
>>> 3) Records corresponding to retweets should be removed
>
>>*Tidiness issue:*
>
>>>In `twitter_archive_df`, the following columns should be combined into one (as each variable should be represented in a single column):
>
>>>>   i. `doggo`
>
>>>>   ii. `floofer`
>
>>>>   iii. `pupper`
>
>>>>   iv. `puppo`

#### 2. image_predictions_df

In [12]:
image_predictions_df.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [13]:
image_predictions_df.tail()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
2070,891327558926688256,https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg,2,basset,0.555712,True,English_springer,0.225770,True,German_short-haired_pointer,0.175219,True
2071,891689557279858688,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,1,paper_towel,0.170278,False,Labrador_retriever,0.168086,True,spatula,0.040836,False
2072,891815181378084864,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,1,Chihuahua,0.716012,True,malamute,0.078253,True,kelpie,0.031379,True
2073,892177421306343426,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,1,Chihuahua,0.323581,True,Pekinese,0.090647,True,papillon,0.068957,True
2074,892420643555336193,https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg,1,orange,0.097049,False,bagel,0.085851,False,banana,0.076110,False


In [14]:
image_predictions_df.sample(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
2013,879376492567855104,https://pbs.twimg.com/media/DDQsQGFV0AAw6u9.jpg,1,tricycle,0.663601,False,Labrador_retriever,0.033496,True,Pembroke,0.018827,True
1711,818536468981415936,https://pbs.twimg.com/media/C1wGkYoVQAAuC_O.jpg,1,swing,0.999403,False,Welsh_springer_spaniel,0.000062,True,bow,0.000030,False
956,705428427625635840,https://pbs.twimg.com/media/CcovaMUXIAApFDl.jpg,1,Chihuahua,0.774792,True,quilt,0.073079,False,Pembroke,0.022365,True
37,666454714377183233,https://pbs.twimg.com/media/CT-5Bs-WUAA2JeC.jpg,1,dalmatian,0.278954,True,Labrador_retriever,0.237612,True,Great_Pyrenees,0.171106,True
1273,750026558547456000,https://pbs.twimg.com/media/CmieRQRXgAA8MV3.jpg,1,standard_poodle,0.258732,True,teddy,0.130760,False,toy_poodle,0.071726,True


>*From the visual assessment performed by scrolling through select records of the dataframe, the following are the issue(s) identified:*
>
>>*Quality issue:*
>
>>>The breeds of the dogs predicted in `p1`, `p2` and `p3` do not follow any standard naming (have underscores, lower case)
>
>>*Tidiness issue:*
>
>>>The `image_prediction_df` dataframe can be joined with the `twitter_archive_df`based on the tweet ID that is common for the two dataframes

#### 3. additional_data_df

In [15]:
additional_data_df.head()

,tweet_id,retweet_count,favourites_count,followers_count,created_at
0,892420643555336193,7492,145955,8876619,Tue Aug 01 16:23:56 +0000 2017
1,892177421306343426,5559,145955,8876619,Tue Aug 01 00:17:27 +0000 2017
2,891815181378084864,3681,145955,8876619,Mon Jul 31 00:18:03 +0000 2017
3,891689557279858688,7661,145955,8876619,Sun Jul 30 15:58:51 +0000 2017
4,891327558926688256,8273,145955,8876619,Sat Jul 29 16:00:24 +0000 2017


In [16]:
additional_data_df.tail()

,tweet_id,retweet_count,favourites_count,followers_count,created_at
2326,666049248165822465,40,145955,8876644,Mon Nov 16 00:24:50 +0000 2015
2327,666044226329800704,125,145955,8876644,Mon Nov 16 00:04:52 +0000 2015
2328,666033412701032449,39,145955,8876644,Sun Nov 15 23:21:54 +0000 2015
2329,666029285002620928,41,145955,8876644,Sun Nov 15 23:05:30 +0000 2015
2330,666020888022790149,454,145955,8876644,Sun Nov 15 22:32:08 +0000 2015


In [17]:
additional_data_df.sample(5)

,tweet_id,retweet_count,favourites_count,followers_count,created_at
696,784057939640352768,11032,145955,8876625,Thu Oct 06 15:49:14 +0000 2016
1437,694925794720792577,891,145955,8876633,Wed Feb 03 16:49:55 +0000 2016
1102,729854734790754305,1088,145955,8876626,Tue May 10 02:05:03 +0000 2016
812,768473857036525572,3364,145955,8876626,Wed Aug 24 15:43:39 +0000 2016
1178,716439118184652801,203,145955,8876628,Sun Apr 03 01:36:11 +0000 2016


>*From the visual assessment performed by scrolling through select records of the dataframe, the following are the issue(s) identified:*
>
>>*Quality issue:*
>
>>>In `additional_data_df`, the `created_at` field is not in datetime format
>
>>*Tidiness issue:*
>
>>>The `additional_data_df` dataframe can be joined with the `twitter_archive_df` and `image_prediction_df` based on the tweet ID that is common for the three dataframes

##### Programmatic assessement

#### 1. twitter_archive_df

In [18]:
# getting the basic information including missing values and data types of the fields in the dataframe
twitter_archive_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

>**Quality Issue(s):**
>
>1. The `timestamp` and `retweeted_status_timestamp` are not in datetime format
>
>2. The fields `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id` and `retweeted_status_user_id` should be strings
>
>3. `expanded_urls` column has some missing values
>
>4. `tweet_id` should not an integer since numerical operations will not be performed

In [19]:
# getting the stats for the numerical fields in the dataframe
twitter_archive_df.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


In [20]:
twitter_archive_df['rating_denominator'].unique()

array([ 10,   0,  15,  70,   7,  11, 150, 170,  20,  50,  90,  80,  40,
       130, 110,  16, 120,   2])

In [21]:
twitter_archive_df['rating_numerator'].unique()

array([  13,   12,   14,    5,   17,   11,   10,  420,  666,    6,   15,
        182,  960,    0,   75,    7,   84,    9,   24,    8,    1,   27,
          3,    4,  165, 1776,  204,   50,   99,   80,   45,   60,   44,
        143,  121,   20,   26,    2,  144,   88])

>**Quality Issue(s):**
>
>1. Looking at the minimum, maximum and other quartile values for numerator and denominator ratings, it looks like they are incorrect in some records
>
>2. `tweet_id` should not an integer since numerical operations will not be performed

In [22]:
# checking if there are any duplicate records in the dataframe
twitter_archive_df.duplicated().sum()

0

#### 2. image_prediction_df

In [23]:
# getting the basic information including missing values and data types of the fields in the dataframe
image_predictions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


>**Quality Issue(s):**
>
>1. There are some missing records in the dataframe since the `twitter_archive_df` dataframe has 2356 records.This issue should be revisited after the dataframes are merged. 
>
>2. `tweet_id` should not an integer since numerical operations will not be performed

In [24]:
# checking if there are any duplicate records in the dataframe
image_predictions_df.duplicated().sum()

0

#### 3. additional_data_df

In [25]:
# getting the basic information including missing values and data types of the fields in the dataframe
additional_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2331 entries, 0 to 2330
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   tweet_id          2331 non-null   int64 
 1   retweet_count     2331 non-null   int64 
 2   favourites_count  2331 non-null   int64 
 3   followers_count   2331 non-null   int64 
 4   created_at        2331 non-null   object
dtypes: int64(4), object(1)
memory usage: 91.2+ KB


In [26]:
# checking if there are any duplicate records in the dataframe
additional_data_df.duplicated().sum()

0

>**Quality Issue(s):**
>
>There are some missing records in the dataframe since the `twitter_archive_df` dataframe has 2356 records (The missing 25 tweets had IDs that were not valid as discussed during data gathering). This issue should be revisited after the dataframes are merged. 

To summarize, the following are the quality and tidiness issues found in the data:

### Quality
##### `twitter_archive` table
- The following columns have missing values: `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id`, `expanded_urls`, and `retweeted_status_timestamp`
- Some dog names (`name`) are not valid (Eg. a, None)
- The `timestamp` and `retweeted_status_timestamp` are not in datetime format
- Incorrect `rating_numerator` and `rating_denominator` values
- The fields `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id` and `retweeted_status_user_id` should be strings
- Records corresponding to retweets should be removed

##### `image_predictions` table
- Predicted breed of dogs in in `p1`, `p2` and `p3` do not follow any standard naming (have underscores, lower case)

##### `additional_data` table
- The `created_at field` is not in datetime format

##### `twitter_archive`, `image_predictions` & `additional_data` tables 
- `tweet_id` should not an integer since numerical operations will not be performed on it
- There are some missing records in `image_predictions` (2075 records) and `additional_data_df` (2331 records) when compared to the `twitter_archive_df` (2356 records). This should be revisited after merging the dataframes.

### Tidiness
- The three dataframes should be merged into one since "Each type of observational unit forms a table"
- The four columns in image_predictions_df dataframe - `doggo`, `floofer`, `pupper`, and `puppo` - should be combined into a single field since "Each variable forms a column"

>*The issues associated with the data are now assessed and segregated into quality and tidiness issues. The next step is to clean the data programmatically.*

<a id='clean'></a>
## Cleaning

>There are 2 types of cleaning. They are:
>
>>1. Manual (not recommended unless the issues are one-off occurrences)
>
>>2. Programmatic
>
>The programmatic data cleaning process includes 3 steps. They are:
>
>>1. Define: convert our assessments into defined cleaning tasks. These definitions also serve as an instruction list so others (or yourself in the future) can look at your work and reproduce it.
>
>>2. Code: convert those definitions to code and run that code.
>
>>3. Test: test your dataset, visually or with code, to make sure your cleaning operations worked.
>
>It is recommended to always make copies of the original pieces of data before cleaning.

### Missing Data

##### Define

##### Code

##### Test

### Tidiness

##### Define

##### Code

##### Test

### Quality

##### Define

##### Code

##### Test

<a id='analyze&visualize'></a>
## Analyze and Visualize

<a id='conclusion'></a>
## Conclusion

>*CONCLUDING REMARKS:*

>*REFERENCES:*